In [1]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [2]:
## Caesar cypher, cyclically shift each letter by an amount k, specified by the key
## c = (p + k)mod 26   to encrypt (right shift)
## p = (c - k)mod 26   to decrypt (left shift)
## 
## there are only 26 possible distinct keys (from 0 to 25), this count includes the key that maps
## each letter onto itself, with a zero shift
def caesar(text, k):
    """
    text is a string, the message to encrypt or decrypt
    k is an int, the shift (can be positive or negative)
    """

    text = text.upper()
    k = k%26

    print(f"the text is {text}")
    print(f"the shift is {k}")

    result = ""
    for c in text:
        i = ALPHABET.index(c) + k
        i = i%26

        result = result + ALPHABET[i]

    return result


In [18]:
caesar("QNGWFWD", -5)

the text is QNGWFWD
the shift is 21


'LIBRARY'

In [5]:
## monoalphabetic cypher: key is a permutation of alphabet, eg DKVQFIBJWPESCXHTMYAUOLRGZN 
## now there are 26! possible keys
## but the frequency distribution of english is retained
## 

def monoalphabetic(text, encrypt=True, KEY="DKVQFIBJWPESCXHTMYAUOLRGZN"):
    """
    text is a string, the message to encrypt, 
    KEY is a 26-character uppercase string, permutations of the ALPHABET

    encrypt is true if you want to encrypt, otherwise False
    
    """

    text = text.upper()
    print(f"the text is {text}")

    source_KEY = ALPHABET
    target_KEY = KEY

    ## invert the keys if you want to decrypt
    if encrypt==False:
        source_KEY, target_KEY = target_KEY, source_KEY 

    print(f"the KEY mappings are:\n{source_KEY}\n{target_KEY}")

    result = ""
    for c in text:
        i = source_KEY.index(c) 

        result = result + target_KEY[i]

    return result

In [7]:
monoalphabetic("AOVVFAA", encrypt=False)

#monoalphabetic("jfssh", source_KEY="DKVQFIBJWPESCXHTMYAUOLRGZN", target_KEY=ALPHABET )

the text is AOVVFAA
the KEY mappings are is:
DKVQFIBJWPESCXHTMYAUOLRGZN
ABCDEFGHIJKLMNOPQRSTUVWXYZ


'SUCCESS'

In [9]:
def vigenere(text, KEY, encrypt=True):
    """
    text is a string, the message we want to encrypt
    KEY is a string of upper case letters representing the vigenere key 
    
    """
    text = text.upper()
    print(f"the text is {text}")
    print(f"the KEY is {KEY}")
    
    ## convert KEY to a list of shifts
    shifts = []
    for c in KEY:
        shifts.append(ALPHABET.index(c))

    keylen = len(shifts)


    
    ## now encrypt text with this KEY
    result = ""
    for c in text:
        i = ALPHABET.index(c)
        shift = shifts[i%keylen]         # cyclically get the shift we want to apply to the ith character in the string

        if not encrypt:
            shift = -shift               # invert shift if decrypting
        
        i_new = (i+shift)%26
        result.append(ALPHABET[i_new])
    
    return result


In [8]:
a = 2
a = -a
print(a)

    


-2


In [1]:
(-3)%26

23